In [1]:
import numpy as np
from scipy.linalg import norm

def EDGES(X, Y, Z, L1, lambda1, lambda2, theta1, theta2, tol, d, iterMax):
    """
    EDGES: Joint decomposition model for spatial transcriptomics and scRNA-seq integration.

    Parameters:
        X      : ST data matrix (shared genes × spatial cells)
        Y      : Shared scRNA-seq data matrix (shared genes × single cells)
        Z      : Unique scRNA-seq data matrix (unique genes × single cells)
        L1     : Normalized graph Laplacian (spatial adjacency constraint)
        lambda1: Graph Laplacian regularization weight
        lambda2: Sparsity regularization weight
        theta1 : Weight for ST reconstruction loss
        theta2 : Weight for unique scRNA-seq loss
        tol    : Tolerance for convergence
        d      : Number of latent factors
        iterMax: Maximum number of iterations

    Returns:
        W1: Shared gene factor matrix
        W2: Unique gene factor matrix (scRNA-seq-specific)
        H1: ST cell embeddings
        H2: scRNA-seq cell embeddings
        p1: Denoised ST data matrix
        p2: Predicted expression matrix for ST cells
    """
    # Initialize matrix shapes
    row_W1 = X.shape[0]
    row_W2 = Z.shape[0]
    col_H1 = X.shape[1]
    col_H2 = Y.shape[1]
    
    # Random initialization with fixed seed 
    rng = np.random.RandomState(seed=42)
    W1 = rng.rand(row_W1, d)
    W2 = rng.rand(row_W2, d)
    H1 = rng.rand(d, col_H1)
    H2 = rng.rand(d, col_H2)

    e1d = np.ones((1, d))

    # Compute initial loss before starting iterations
    loss_X1 = norm(X - W1 @ H1, 'fro')**2
    loss_X2 = norm(Y - W1 @ H2, 'fro')**2
    loss_X3 = norm(Z - W2 @ H2, 'fro')**2
    loss_H1 = np.trace(H1 @ L1 @ H1.T)
    loss_H = e1d @ (H1 @ H1.T) @ e1d.T + e1d @ (H2 @ H2.T) @ e1d.T

    delta_init1 = theta1 * loss_X1 + loss_X2 + theta2 * loss_X3 + lambda1 * loss_H1 + lambda2 * loss_H
    delta2 = delta_init1

    # Main optimization loop
    for iter in range(iterMax):
        # Update W1
        X1H1t = X @ H1.T
        X2H2t = Y @ H2.T
        W1H1H1t = W1 @ (H1 @ H1.T)
        W1H2H2t = W1 @ (H2 @ H2.T)
        W1 *= (theta1 * X1H1t + X2H2t) / (theta1 * W1H1H1t + W1H2H2t + 1e-8)

        # Update H1
        W1tX1 = W1.T @ X
        W1tW1H1 = W1.T @ W1 @ H1
        eddH1 = (e1d.T @ e1d) @ H1
        H1L1 = H1 @ L1
        H1 *= (theta1 * W1tX1) / (theta1 * W1tW1H1 + lambda2 * eddH1 + lambda1 * H1L1 + 1e-8)

        # Update W2
        X3H2t = Z @ H2.T
        W2H2H2t = W2 @ (H2 @ H2.T)
        W2 *= X3H2t / (W2H2H2t + 1e-8)

        # Update H2
        W1tX2 = W1.T @ Y
        W2tX3 = W2.T @ Z
        W1tW1H2 = W1.T @ W1 @ H2
        W2tW2H2 = W2.T @ W2 @ H2
        eddH2 = (e1d.T @ e1d) @ H2
        H2 *= (W1tX2 + theta2 * W2tX3) / (W1tW1H2 + theta2 * W2tW2H2 + lambda2 * eddH2 + 1e-8)

        # Compute reconstruction loss
        loss_X1 = norm(X - W1 @ H1, 'fro')**2
        loss_X2 = norm(Y - W1 @ H2, 'fro')**2
        loss_X3 = norm(Z - W2 @ H2, 'fro')**2
        loss_H1 = np.trace(H1 @ L1 @ H1.T)
        loss_H = e1d @ (H1 @ H1.T) @ e1d.T + e1d @ (H2 @ H2.T) @ e1d.T

        total_loss = (theta1 * loss_X1 + loss_X2 + theta2 * loss_X3 +
                      lambda1 * loss_H1 + lambda2 * loss_H)

        # Convergence check using relative change in loss
        stop_value = abs((delta2 - total_loss) / (delta_init1 - total_loss + 1e-8))
        if stop_value < tol:
            print(f"Converged at iteration {iter+1}")
            break

        delta2 = total_loss

    # Compute final outputs
    p1 = W1 @ H1  # Denoised ST matrix
    p2 = W2 @ H1  # Predicted expression matrix from unique scRNA-seq

    return W1, W2, H1, H2, p1, p2


In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import normalize
from scipy.sparse import csr_matrix
from scipy.stats import pearsonr

# === Input Laplacian matrix ===
L1 = pd.read_csv('Laplacian.csv')

# === Hyperparameter configuration ===
lambda1 = 10**-5      # Spatial regularization
lambda2 = 10**1       # Sparsity regularization
theta1 = 10**-1       # ST decomposition weight
theta2 = 10**-4       # scRNA-seq decomposition weight
tol = 1e-7            # Convergence tolerance
d = 20                # Latent dimensionality
iterMax = 600         # Maximum iterations

# === Load expression matrices from CSV ===
def load_matrix(path):
    return pd.read_csv(path, header=0, index_col=0).values

X_list = [
    load_matrix('stdata_X11.csv'),
    load_matrix('stdata_X12.csv'),
    load_matrix('stdata_X13.csv')
]
Y_list = [
    load_matrix('RNAdata_X21.csv'),
    load_matrix('RNAdata_X22.csv'),
    load_matrix('RNAdata_X23.csv')
]
Z_list = [
    load_matrix('RNAdata_X31.csv'),
    load_matrix('RNAdata_X32.csv'),
    load_matrix('RNAdata_X33.csv')
]
pre_list = [
    load_matrix('stdata_pre1.csv'),
    load_matrix('stdata_pre2.csv'),
    load_matrix('stdata_pre3.csv')
]

# === Run EDGES for each fold and collect inferred results ===
st_inferr_fold = []
for i in range(3):
    print(f"Processing group {i+1}...")
    X, Y, Z = X_list[i], Y_list[i], Z_list[i]
    _, _, _, _, p1, p2 = EDGES(X, Y, Z, L1, lambda1, lambda2, theta1, theta2, tol, d, iterMax)

    if p2.shape[0] < 2001:
        raise ValueError(f"Fold {i+1} p2 has fewer than 2001 rows")

    st_inferr_fold.append(p2[2000:, :])  # Slice from row index 2000 onward

# === Concatenate all folds and evaluate ===
st_infer = np.vstack(st_inferr_fold)
st_raw = np.vstack(pre_list)

# === Compute Pearson correlation coefficients between predicted and raw ST data ===
pccs = [pearsonr(st_infer[i], st_raw[i])[0] for i in range(st_raw.shape[0])]
print("Mean PCC:", np.mean(pccs))


Processing group 1...
Converged at iteration 507
Processing group 2...
Converged at iteration 339
Processing group 3...
Converged at iteration 564
Mean PCC: 0.4530917279449207
